In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.layers import Dense
#splitting data into test-train 
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from nltk.stem.porter import PorterStemmer
import nltk
from nltk.tokenize import word_tokenize
from keras.utils import to_categorical
from sklearn.feature_extraction.text import CountVectorizer
from keras import models
from keras import layers
from sklearn.preprocessing import LabelEncoder

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import re
import os
print(os.listdir("../input"))


# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


['exercise-1', 'bertpretrained']


In [2]:
#Data loading
data = pd.read_csv("../input/exercise-1/text_emotion.csv")

In [3]:
#converting column from data frame into list
y = data["sentiment"].tolist()

In [4]:
x = data["content"].tolist()

In [5]:
#test_train split
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.20)

In [6]:
import keras
from sklearn import preprocessing
import pandas as pd
from sklearn.preprocessing import LabelEncoder

#label encoding   
le = LabelEncoder() 
ytrain_enc = le.fit_transform(ytrain)
ytest_enc = le.transform(ytest)
ytrain_enc = keras.utils.to_categorical(ytrain_enc, num_classes=13)
ytest_enc = keras.utils.to_categorical(ytest_enc, num_classes=13)
    

In [ ]:
np.unique(ytest_enc)

In [7]:
#Install keras-bert library
!pip install -q keras-bert
import keras_bert

In [8]:
# Constants

SEQ_LEN = 32
BATCH_SIZE = 64
EPOCHS = 5
LR = 1e-6

In [9]:
# Environment
import os

pretrained_path = '../input/bertpretrained/uncased_l-24_h-1024_a-16/uncased_L-24_H-1024_A-16/'
config_path = os.path.join(pretrained_path, 'bert_config.json')
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

# TF_KERAS must be added to environment variables in order to use TPU
os.environ['TF_KERAS'] = '1'

In [10]:
# Load Basic Model
import codecs
from keras_bert import load_trained_model_from_checkpoint

token_dict = {}
with codecs.open(vocab_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)

model = load_trained_model_from_checkpoint(
    config_path,
    checkpoint_path,
    training=True,
    trainable=True,
    seq_len=SEQ_LEN,
)

In [11]:
# Preprocessing of the data and Conversion of Data to Array
import re
import os
import numpy as np
from tqdm import tqdm
from keras_bert import Tokenizer

tokenizer = Tokenizer(token_dict)    

def preprocess_tweet(tweet):  #Removing special characters and digits
    words = tweet.split(" ")
    new_list = []
    for word in words:
        if "@" in word or "http" in word:
            pass
        else:
            new_list.append(word)
    new_tweet = " ".join(new_list)
    tweet1 = re.sub(r"@[^\s]+[\s]?",'',new_tweet)
    tweet3 = re.sub('[0-9]', '', tweet1)
    return tweet3

def load_data(x):   # Defining the function
    global tokenizer 
    indices = [] 
    for text in tqdm(x):   #For content in a list, apply for loop
        text = preprocess_tweet(text)
        ids,segments = tokenizer.encode(text, max_len=SEQ_LEN)
        indices.append(ids)
        
    indices = np.array(indices) #creating an array of indices
    return [indices, np.zeros_like(indices)]#returning two arrays,one of indices 
                                           #and a similar one of zeros  

#Applying function on xtrain and xtest (lists of content)       
test_x = load_data(xtest)   
train_x = load_data(xtrain)        

100%|██████████| 32000/32000 [00:05<00:00, 5652.66it/s]


In [12]:
# Build Custom Model
import keras
from keras_bert import AdamWarmup, calc_train_steps
from keras.layers import Dense

inputs = model.inputs[:2]
dense = model.get_layer('NSP-Dense').output
outputs = Dense(units=13, activation='softmax')(dense)

decay_steps, warmup_steps = calc_train_steps(
    ytrain_enc.shape[0],
    batch_size=BATCH_SIZE,
    epochs=10,
)

model = keras.models.Model(inputs, outputs)
model.compile(
    AdamWarmup(decay_steps=decay_steps, warmup_steps=warmup_steps, lr=LR),
    loss='categorical_crossentropy',
    metrics=['accuracy'])


In [13]:
# Initialize Variables
import tensorflow as tf
import keras.backend as K

sess = K.get_session()
uninitialized_variables = set([i.decode('ascii') 
for i in sess.run(tf.report_uninitialized_variables())])
    init_op = tf.variables_initializer(
    [v for v in tf.global_variables() if v.name.split(':')[0] in uninitialized_variables]
)
sess.run(init_op)

In [14]:
# with tf.keras.utils.custom_object_scope(get_custom_objects()):
        
model.fit(
        train_x,
        ytrain_enc,
        epochs = 5,
        batch_size = BATCH_SIZE
)

Epoch 1/5
32000/32000 [==============================] - 453s 14ms/step - loss: 2.5972 - acc: 0.1293
Epoch 2/5
32000/32000 [==============================] - 416s 13ms/step - loss: 2.0678 - acc: 0.2869
Epoch 3/5
32000/32000 [==============================] - 416s 13ms/step - loss: 1.9529 - acc: 0.3331
Epoch 4/5
32000/32000 [==============================] - 416s 13ms/step - loss: 1.8930 - acc: 0.3536
Epoch 5/5
32000/32000 [==============================] - 416s 13ms/step - loss: 1.8616 - acc: 0.3656


In [ ]:
model.evaluate(
        test_x,
        ytest_enc,
        batch_size = BATCH_SIZE
)

In [ ]:
model.optimizer.lr = 5e-7

In [ ]:
model.optimizer.get_config()